In [20]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import shutil
import os
import csv
import sys

import json
import datetime
import pandas as pd
import numpy as np
import talib as ta
import math

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

#import trader
import schedule
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split

#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [((datetime.datetime.fromisoformat(res['time'][:19]))+datetime.timedelta(hours=+9)).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

def logarism(x):
    x = abs(x)
    return math.log(x)


def get_now_data(time, period_hours, money, foot, back_hours = 0, times = 0):
    global BACK
    all_data = []
    #time = time.replace(hour=7, hour=0, second=0, microsecond=0)
    #入力された日の当日の7時
    
    NY = datetime.timedelta(hours=-14)
    back = datetime.timedelta(hours=-back_hours)
    
    force = datetime.timedelta(hours=-BACK)
    
    date_to = time + NY + datetime.timedelta(seconds=-1) + back + force

    date_from = date_to + datetime.timedelta(hours=-period_hours)
#     for i in range(times+1):    
    ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加

    date_to = date_from
    date_from = date_to + datetime.timedelta(hours=-period_hours)
#         print(date_from, date_to)

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)

#     print(date_from, date_to)
#     print(df)
    
    df.columns = ['Datetime','Volume', 'Open', 'High', 'Low', 'Close']
    
    df['Volume'] = df['Volume'].astype('double')
    df['Open'] = df['Open'].astype('double')
    df['Close'] = df['Close'].astype('double')
    df['High'] = df['High'].astype('double')
    df['Low'] = df['Low'].astype('double')
    
    
    close = np.array(df["Close"]).astype(float)
#     #以下、talibを用いてテクニカル指標（今回の学習で用いる特徴量）を算出しdf_feature入れる
#     #単純移動平均は、単純移動平均値とその日の終値の比を特徴量として用いる
    df["SMA2"]= ta.EMA(close, timeperiod=2) / close
#     df["SMA3"]= ta.SMA(close, timeperiod=3) / close
#     df["SMA4"]= ta.SMA(close, timeperiod=4) / close
#     df["SMA5"]= ta.EMA(close, timeperiod=5)# / close
#     df["SMA6"]= ta.SMA(close, timeperiod=6) / close
#     df["SMA7"]= ta.SMA(close, timeperiod=7) / close
#     df["SMA8"]= ta.SMA(close, timeperiod=8) / close
#     df["SMA9"]= ta.SMA(close, timeperiod=9) / close
#     df["SMA10"]= ta.EMA(close, timeperiod=10) / close
    df["RSI"] = ta.RSI(close, timeperiod=9) / close
    #ボリンジャーバンド Main 
    upper, middle, lower = ta.BBANDS(close, timeperiod=10, nbdevup=2, nbdevdn=2)
    df["BBANDS+σ"] = upper / close * 1000
    df["BBANDS-σ"] = lower / close * 1000
    
    #df['Rate']=(df['Close']-df['Open'])/df['Open']
    
#     df['Open'] = df['Open'].apply(lambda x: logarism(x))
#     df['Close'] = df['Close'].apply(lambda x: logarism(x))
    
    df["SMA2"]= df["SMA2"].apply(lambda x: logarism(x)) * 1000
#     df["SMA3"]= df["SMA3"].apply(lambda x: logarism(x)) * 1000
#     df["SMA4"]= df["SMA4"].apply(lambda x: logarism(x)) * 1000
#     df["SMA5"]= df["SMA5"].apply(lambda x: logarism(x)) * 1000
#     df["SMA6"]= df["SMA6"].apply(lambda x: logarism(x)) * 1000
#     df["SMA7"]= df["SMA7"].apply(lambda x: logarism(x)) * 1000
#     df["SMA8"]= df["SMA8"].apply(lambda x: logarism(x)) * 1000
#     df["SMA9"]= df["SMA9"].apply(lambda x: logarism(x)) * 1000
#     df["SMA10"]= df["SMA10"].apply(lambda x: logarism(x)) * 1000
    
    df["MACD"], _ , _= ta.MACD(close, fastperiod=3, slowperiod=18, signalperiod=9)/close
    
    
#     df['Rate']=(df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')
    
#     df['Rate'] = df['Rate']*1000
#     df['true']=(df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')
    
    df['class'] = ((df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')).apply(lambda x: classify_two(x))
    
#     print(df.head(1))
#     df_next = df.tail(1)
    
    df = df.dropna()
    
#     df['Open'] = df['Open']/df['Close']
    
    df = df.drop('Open', axis=1)
    df = df.drop('High', axis=1)
    df = df.drop('Low', axis=1)
    df = df.drop('Close', axis=1)
    df = df.drop('Volume', axis=1)
    
    df = df.set_index('Datetime')
    
    return df



def get_csv(back_hours, hours, money):
    global MULTI
    time = datetime.datetime.now() 
    
    df = get_now_data(time, period_hours=hours, money=money, foot = FOOT , back_hours=back_hours, times=MULTI)
    date_to = (time+datetime.timedelta(hours=-back_hours)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(hours=-hours)+datetime.timedelta(hours=-back_hours)).strftime("%Y-%m-%d")
    df.to_csv("csv/"+str(money)+"_"+ date_from + "_" + date_to +"_"+str(hours)+'.csv', mode = 'w')
    print('"'+str(money)+"_"+ date_from + "_" + date_to +"_"+str(hours*MULTI)+'.csv', end='", ')
    
    return df

def get_moneys_csv(back_hours, hours, moneylist):
    csvlist = []
    
    time = datetime.datetime.now()
    date_to = (time+datetime.timedelta(hours=-back_hours)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(hours=-hours)+datetime.timedelta(hours=-back_hours)).strftime("%Y-%m-%d")
    
    for i in range (len(moneylist)):
        get_csv(back_hours, hours, money=moneylist[i])
        csvlist.extend([str(moneylist[i])+"_"+ date_from + "_" + date_to +"_"+str(hours)+'.csv'])
    
    return csvlist
        
def classify_two(x):
        if x<0:
            return -1
        elif 0<=x:
            return 1
        
def classify_random(x):
        ram = random.randint(0,1)
        if ram==0:
            return -1
        elif ram==1:
            return 1
        

def get_int_csv(moneylist):
    global TERM, DNUM
    shutil.rmtree('csv')
    os.mkdir('csv')
#     array = get_moneys_csv(back_hours=TERM, hours=DNUM, moneylist = moneylist)
    array = get_moneys_csv(back_hours=0, hours=DNUM, moneylist = moneylist)
    return array

def make_bins(max_num, min_num, bin_num):
    global DIVIDE
    delta = max_num-min_num
    delta = delta/(DIVIDE-1)
    bins = []
    povit = min_num
    for i in range(DIVIDE-1):
        povit += delta
        bins.extend([povit])
#     print(bins)
    return bins

def digitize_data(df):
    bin_num = len(df)
    for i in range(len(df.columns)):
        col = df.columns.values.tolist()[i]
        if col == "class":
            continue
        elif col == "true":
            continue
        else:
            max_num = df[col].max()
            min_num = df[col].min()
#             max_num = df[col].quantile(0.75)
#             min_num = df[col].quantile(0.25)
            df[col] = np.digitize(df[col], make_bins(max_num, min_num, bin_num))
    return df

def grider(dataname):#csv data only
    df = pd.read_csv("csv/"+dataname, index_col='Datetime')
    df['class'] = df['class'].shift(-1)
    df = digitize_data(df)
    df = df.dropna()
    return df
    
def make_table(line, money):
    global DIVIDE
    count = 0
    df = grider(money)
    print("Table data")
    print(df.head(1).index.values, end=' to ')
    print(df.tail(1).index.values)

#     table = np.zeros((DIVIDE,DIVIDE,DIVIDE,DIVIDE,DIVIDE,DIVIDE))
    table = np.zeros((DIVIDE,DIVIDE,DIVIDE,DIVIDE,DIVIDE))
    for y in range(len(df)):
        temp = []
        for x in range(len(df.columns)-1):
            temp.extend([df.iat[y,x]])
#         table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] += int(df.iat[y,len(df.columns)-1])
        table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]] += int(df.iat[y,len(df.columns)-1])
        count += 1
#     print("Beyond LINE: ", end='')
#     print(np.count_nonzero(table>line)/np.count_nonzero(table!=0))
#     print("table count: "+str(count))
    return table

def buy_test(table, money):
    global FOOT, TERM, LINE, POSSESTION, PAY, PAYOUT, WIN, LOSE, COUNT
    df_now = get_now_data(datetime.datetime.now(), TERM, money, FOOT, back_hours = 0, times = 0)
#     df_now['class'] = df_now['class'].shift(1)
    df_now = digitize_data(df_now)
    df_now = df_now.dropna()
    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    count = [0,0,0]
    win = 0
    lose = 0
    glaph = []
    pos = POSSESTION
    history = []
    for y in range(len(df_now)):
        temp = []
        for x in range(len(df_now.columns)-1):
            temp.extend([df_now.iat[y,x]])
#         value = table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]]
        value = table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]]
        if value>=LINE: #HIGH予想
            count[0] += 1
            pos -= PAY
            if(int(df_now.iat[y,len(df_now.columns)-1])>0): #実際HIGHなら
                pos += PAY*PAYOUT
                win += 1
                history.extend([0.9])
            else:
                lose += 1
                history.extend([-1])
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] = 0 #never use this cell
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] = -1*table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]]
                table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]] = 0 #never use this cell
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]] = -1*table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]]
                continue
        elif value<=-LINE: #LOW予想
            count[1] += 1
            pos -= PAY
            if(int(df_now.iat[y,len(df_now.columns)-1])<0): #実際LOWなら
                pos += PAY*PAYOUT
                win += 1
                history.extend([0.9])
            else:
                lose += 1
                history.extend([-1])
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] = 0 #never use this cell
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] = -1*table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]]
                table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]] = 0 #never use this cell
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]] = -1*table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]]
                continue
        else:
            count[2] += 1
            history.extend([0])
        glaph.extend([pos])
    print("HIGH/LOW/STAY")
    print(count)
    print("win: "+str(win))
    print("lose: "+str(lose))
    
    WIN += win
    LOSE += lose
    COUNT += sum(count)
    
    if(win+lose==0):
        print("No Trade")
    else:
        print("Rate: "+str(win/(win+lose)))
    print("Posesstion: "+str(pos))
    POSSESTION += pos - POSSESTION
    
#     POSSESTION = 10000
    
#     if(POSSESTION < 0):
#         sys.exit()
#     plt.plot(list(range(0,len(glaph))), glaph)
#     plt.show()

#     for k in range(len(df_next.columns)-1):
#             temp.extend([df_next.iat[1,k]])
#     value = table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]]
#     print(value)
    return win,lose, history

def RFtest(money):
    df = grider(money)
    print(df)
    df_y = df["class"]
    df_X = df.drop("class", axis=1)
    X_train, X_test, y_train, y_test = train_test_split(df_X, df_y)
    
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    result = clf.predict(X_test)
    precision = precision_score(y_test, result, average=None)
    accuracy = accuracy_score(y_test, result)
    confusion = confusion_matrix(y_test, result)
    
    print(accuracy)



def trade(table, money):
    global FOOT
    
    df_now = get_now_data(datetime.datetime.now(), 100, money, FOOT, back_hours = 0, times = 0)
    df_now = digitize_data(df_now)
    df_now['class'] = df_now['class'].shift(-1)
    df_next = df_now.tail(1)
    print(df_next.tail(1).index.values)
#     df_now = df_now.dropna()
    temp = []
    for x in range(len(df_next.columns)-1):
        temp.extend([df_next.iat[0,x]])
    value = table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]]
    if value>=LINE: #HIGH予想
        print("HIGH ENTRY")
        trader.select_money(money)
        trader.high_entry()
    elif value<=-LINE: #LOW予想
        print("LOW ENTRY")
        trader.select_money(money)
        trader.low_entry()
    else:
        print("STAY")
    


def main():
    global LINE, POSSESTION
    array = get_int_csv(MONEYS)
    w = 0
    l = 0
    for i in range(len(array)):
        print("")
        print("-----------------------")
        print(MONEYS[i])
        table = make_table(LINE, array[i])
#         print(table)
        win, lose, history = buy_test(table, MONEYS[i])
        w += win
        l += lose
#         with open('result.csv', 'a') as f:
#             writer = csv.writer(f)
#             writer.writerow([MONEYS[i], history])
    print("RESULT: "+str(POSSESTION))
    RESULT.extend([POSSESTION])
    with open('result.csv', 'a') as f:
        writer = csv.writer(f)
#         writer.writerow([money, pos])
        if(w+l!=0):
            writer.writerow([POSSESTION, w/(w+l)])
        else:
            writer.writerow([POSSESTION])
    

FOOT = 'H1'
DNUM = 100
MULTI = 1  #datanum = DNUM * MULTI
DIVIDE = 3  #GRIDの数
LINE = 1  #トレードのライン(LINE以上，以下の時エントリー)
TERM = 100  #テスト期間
PAY = 1000  #支払額
PAYOUT = 1.9 #ペイアウト率
POSSESTION = 10000 #所持金
BACK = 0
RESULT = []
WIN = 0
LOSE = 0
COUNT = 0

MONEYS = [
        'AUD_JPY',
    
#         'AUD_NZD',
#         'AUD_USD',
#         'CAD_JPY',
#         'CHF_JPY',
#         'EUR_AUD',
#         'EUR_GBP',
    
        'EUR_JPY',
        'EUR_USD',
    
#         'GBP_AUD',
    
        'GBP_JPY',
    
#         'GBP_USD',
#         'NZD_JPY',
#         'NZD_USD',
#         'USD_CAD',
#         'USD_CHF',
        'USD_JPY'
    ]

# MONEYS = [
#         'AUD_JPY',
#         'AUD_NZD',
#         'AUD_USD',
#         'CAD_JPY',
#         'CHF_JPY',
#         'EUR_AUD',
#         'EUR_GBP',
#         'EUR_JPY',
#         'EUR_USD',
#         'GBP_AUD',
#         'GBP_JPY',
#         'GBP_USD',
#         'NZD_JPY',
#         'NZD_USD',
#         'USD_CAD',
#         'USD_CHF',
#         'USD_JPY'
#     ]

# np.set_printoptions(threshold=np.inf)

# for x in range(100):
#     print("")
#     print("TEST: "+ str(x))
#     main()
#     BACK += TERM
#     print("Trade/Trade chance: "+str(WIN+LOSE)+"/"+str(COUNT))
#     print("Trade rate: "+str((WIN+LOSE)/COUNT))
#     print("Rate: "+str(WIN/(WIN+LOSE)))

# DNUM = 100
# DIVIDE = 2
# BEST = 0
# LINE = 1
# for y in range(5):
#     for i in range(10):
#         for j in range(7):
#             print("DNUM:"+str(DNUM)+" /DIVIDE:"+str(DIVIDE)+ " /LINE:" +str(LINE)+" /BEST:"+str(BEST))
#             for x in range(100):
#                 print("")
#                 print("TEST: "+ str(x))
#                 main()
#                 BACK += TERM
#                 if(LOSE==0 and WIN==0):
#                     LOSE = 1
#                 print("Trade/Trade chance: "+str(WIN+LOSE)+"/"+str(COUNT))
#                 print("Trade rate: "+str((WIN+LOSE)/COUNT))
#                 print("Rate: "+str(WIN/(WIN+LOSE)))
#                 if(POSSESTION<0):
#                     break
#             with open('result.csv', 'a') as f:
#                     writer = csv.writer(f)
#                     if(BEST<WIN/(WIN+LOSE)):
#                         BEST = WIN/(WIN+LOSE)
#                         writer.writerow(["BEST", DNUM, DIVIDE, LINE, WIN/(WIN+LOSE), POSSESTION])
#                     else:
#                         writer.writerow([DNUM, DIVIDE, LINE, WIN/(WIN+LOSE), POSSESTION])
#             WIN = 0
#             LOSE = 0
#             POSSESTION = 10000
#             BACK = 0
#             COUNT = 0
#             DIVIDE += 1
#         DNUM+=100
#         DIVIDE = 2
#         BACK = 0
#     DNUM = 100
#     LINE += 1
        
def job():
    global MONEYS
    array = get_int_csv(MONEYS)
    trader.next_hour()
    for s in range(len(MONEYS)):
        print("\n------------------------")
        print(MONEYS[s])
        table = make_table(LINE, array[s])
        if s == 0:
            time.sleep(10)
        trade(table, MONEYS[s])

def auto():
    schedule.every().day.at("07:00").do(job)
    schedule.every().day.at("08:00").do(job)
    schedule.every().day.at("09:00").do(job)
    schedule.every().day.at("10:00").do(job)
    schedule.every().day.at("11:00").do(job)
    schedule.every().day.at("12:00").do(job)
    schedule.every().day.at("13:00").do(job)
    schedule.every().day.at("14:00").do(job)
    schedule.every().day.at("15:00").do(job)
    schedule.every().day.at("16:00").do(job)
    schedule.every().day.at("17:00").do(job)
    schedule.every().day.at("18:00").do(job)
    schedule.every().day.at("19:00").do(job)
    schedule.every().day.at("20:00").do(job)
    schedule.every().day.at("21:00").do(job)
    schedule.every().day.at("22:00").do(job)
    schedule.every().day.at("23:00").do(job)
    schedule.every().day.at("00:00").do(job)
    
    

    while True:
        schedule.run_pending()
        time.sleep(5)

        
# get_int_csv("USD_JPY")
RFtest("USD_JPY.csv") 
    

# trader.setup_demo()
# job()
# auto()




                  SMA2  RSI  BBANDS+σ  BBANDS-σ  MACD  class
Datetime                                                    
2021-02-02 04:00     1    1         0         1     1    1.0
2021-02-02 05:00     1    1         0         1     1   -1.0
2021-02-02 06:00     1    1         0         1     1    1.0
2021-02-02 07:00     1    1         0         1     1   -1.0
2021-02-02 08:00     1    0         1         2     0    1.0
2021-02-02 09:00     1    0         1         1     0   -1.0
2021-02-02 10:00     1    0         1         1     0    1.0
2021-02-02 11:00     1    0         1         1     0    1.0
2021-02-02 12:00     0    1         0         1     0    1.0
2021-02-02 13:00     0    1         0         1     1   -1.0
2021-02-02 14:00     1    1         0         1     1    1.0
2021-02-02 15:00     0    1         0         1     1   -1.0
2021-02-02 16:00     1    1         0         1     1   -1.0
2021-02-02 17:00     1    0         1         1     0    1.0
2021-02-02 18:00     0  